In [27]:
import os 
import cv2
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split

In [28]:
train_data_path = './dataset/dataset/train'
test_data_path = './dataset/dataset/test'

classes_train = os.listdir(train_data_path)
classes_test = os.listdir(train_data_path)

assert classes_train==classes_test

In [29]:
# train_images_path and test_images_path
def get_image_paths(data_path, classes_train):
    images_paths = []
    for class_ in classes_train:
        image_directory = os.path.join(data_path, class_)
        images_paths = [os.path.join(image_directory, image_name) for image_name in os.listdir(image_directory)]
    return images_paths

In [30]:
train_image_paths = get_image_paths(train_data_path, classes_train)
test_image_paths = get_image_paths(test_data_path, classes_train)

In [31]:
assert all(list(map(lambda filepath: os.path.exists(filepath), train_image_paths)))
assert all(list(map(lambda filepath: os.path.exists(filepath), test_image_paths)))

In [32]:
thresh = int(0.8*len(train_image_paths))
train_img_paths = train_image_paths[:thresh]
valid_img_paths = train_image_paths[thresh:] 

len(train_image_paths), len(train_img_paths), len(valid_img_paths), len(test_image_paths)

(623, 498, 125, 100)

In [33]:
image = cv2.imread(train_img_paths[0])#.shape
image = np.array(image, np.int32)
# image

image = torch.from_numpy(image)
type(image), image.dtype, image.permute(2, 0, 1).shape

(torch.Tensor, torch.int32, torch.Size([3, 480, 640]))

In [34]:
class_to_index = {class_name:index for index, class_name in enumerate(classes_train)}
class_to_index

{'Closed': 0, 'no_yawn': 1, 'Open': 2, 'yawn': 3}

In [35]:
# Create Custom Dataset
class ClassificationDataset(Dataset):
    def __init__(self, image_paths, transform=None) -> None:
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_filepath = self.image_paths[index]
        image = cv2.imread(image_filepath)
        image = np.array(image, np.float32)

        image = torch.from_numpy(image).permute(2, 0, 1)

        label = Path(image_filepath).parts[-2]
        label = class_to_index[label]

        return image, label

In [36]:
train_dataset = ClassificationDataset(train_img_paths)
validation_dataset = ClassificationDataset(valid_img_paths)
test_dataset = ClassificationDataset(test_image_paths)

In [37]:
batch_size = 16
num_clients = 6
records_per_split = int(len(train_dataset) / num_clients) 

train_dataset_split = random_split(
    train_dataset,
    [records_per_split] * num_clients
)

In [38]:
train_loaders = [
    DataLoader(split, batch_size=batch_size, shuffle=True)
    for split in train_dataset_split
]

valid_loaders = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_loaders = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [39]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [40]:
class ConvBlock(nn.Module):
    def __init__(self, in_c, out_c, k, s, p) -> None: # k, s, p -> Kernel size, s-stride, p-padding
        super().__init__()
        self.conv = nn.Conv2d(in_c, out_c, k, s, p)
        self.pooling = nn.MaxPool2d(2)

    def forward(self, x):
        # return F.relu(self.conv(x))
        return F.relu(self.pooling(self.conv(x)))


class ClassificationNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = ConvBlock(3, 32, 3, 2, 1)
        self.conv2 = ConvBlock(32, 64, 3, 2, 1)
        self.conv3 = ConvBlock(64, 128, 3, 2, 1)
        self.fc1 = nn.Linear(8960, 128)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [41]:
def client_model_train(client_model, train_loader, optimizer):
    client_model.train()
    for images, labels in train_loader:
        # images, labels = images.cuda(), labels.cuda() # uncomment if using GPU
        optimizer.zero_grad()
        output = client_model(images)
        loss = F.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
    # print(f"{loss.item()}")
    return loss.item()


def server_model_gradient_aggregation(global_model, client_models):
    global_model_state = global_model.state_dict()
    for k in global_model_state.keys():
        global_model_state[k] = torch.stack(
            [client_models[i].state_dict()[k] for i in range(len(client_models))], 0
        ).mean(0)
    global_model.load_state_dict(global_model_state)
    client_models_updated = [model.load_state_dict(global_model.state_dict()) for model in client_models]
    return global_model, client_models_updated
        


def test(global_model, test_loader):
    global_model.eval()
    test_loss, correct_pred = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            # images, labels = images.cuda(), labels.cuda() # uncomment if using GPU
            output = global_model(images)
            test_loss += F.nll_loss(output, labels, reduction='sum').item() # add up loss from each batch
            pred = output.argmax(dim=1, keepdim=True)
            correct_pred += pred.eq(labels.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader)
    acc = correct_pred / len(test_loader)

    return test_loss, acc

In [ ]:
global_model = ClassificationNetwork()
optimizers = [optim.SGD(global_model.parameters(), lr=0.0000001) for _ in range(num_clients)]